In [2]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from itertools import chain
import plotly.figure_factory as ff
import scipy
import dash_bootstrap_components as dbc
import numpy as np 
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ks_2samp
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE

In [3]:
#accepts excel or csv files
#converts file to pandas dataframe
def uploadFile(Path_):
    if Path_.endswith(".xlsx"):
        df = pd.DataFrame(pd.read_excel(Path_))
        df = df.to_csv ("data.csv", index = None, header=True)
        df = pd.DataFrame(pd.read_csv("data.csv"))
    else:
        df = pd.DataFrame(pd.read_csv(Path_))
    return(df)
#file to upload
#df_ = uploadFile("/Users/suziepalmer/Desktop/IsoplexisV9/data.csv")
#df_ = uploadFile("/home2/s180020/Desktop/IsoplexisData/ABX_ICT_TDLN_121221_CD3+_raw_data.csv")
df_ = uploadFile("/home2/s180020/Desktop/IsoplexisData/WT_ABX_MLN_T_Cell_CD4+_CD8+_raw_data.csv")

#this is the original cytokine list
cyto_list = ['BCA-1', 'CCL-11', 'FAS', 'GM-CSF', 'Granzyme B', 
    'IFN-g', 'IL-10', 'IL-12p70', 'IL-13', 'IL-15', 'IL-17A', 
    'IL-18', 'IL-1b', 'IL-2', 'IL-21', 'IL-27', 'IL-4', 
    'IL-5', 'IL-6', 'IL-7', 'IP-10', 'KC', 'MCP-1', 'MIP-1a', 
    'RANTES', 'sCD137', 'TGF-b', 'TNF-a']
#this is the original description list
description_list = ["Donor", "Cell Subset", "Stimulation"]

#this column creates a new column which combines all of the descriptions
df_['Treatment Conditions'] = df_[description_list].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1)

In [3]:
df_col_list = df_[description_list].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1)

df_labels_un = list(df_['Treatment Conditions'].unique())
df_labels_un

['ABX CD4+ stimulated',
 'WT CD4+ stimulated',
 'ABX CD8+ stimulated',
 'WT CD8+ stimulated']

In [4]:
import itertools
#this creates permutations of the list that the user selects to order the graph
permutations = list(itertools.permutations(df_labels_un))

default_permutation = permutations_list[1]

NameError: name 'permutations_list' is not defined

In [6]:
import itertools
df_labels_un = list(df_['Treatment Conditions'].unique())
#this creates permutations of the list that the user selects to order the graph
permutations = list(itertools.permutations(df_labels_un))
permutations_list = []
for i in permutations:
    permutations_list.append(", ".join(i))
permutations_list

['ABX CD4+ stimulated, WT CD4+ stimulated, ABX CD8+ stimulated, WT CD8+ stimulated',
 'ABX CD4+ stimulated, WT CD4+ stimulated, WT CD8+ stimulated, ABX CD8+ stimulated',
 'ABX CD4+ stimulated, ABX CD8+ stimulated, WT CD4+ stimulated, WT CD8+ stimulated',
 'ABX CD4+ stimulated, ABX CD8+ stimulated, WT CD8+ stimulated, WT CD4+ stimulated',
 'ABX CD4+ stimulated, WT CD8+ stimulated, WT CD4+ stimulated, ABX CD8+ stimulated',
 'ABX CD4+ stimulated, WT CD8+ stimulated, ABX CD8+ stimulated, WT CD4+ stimulated',
 'WT CD4+ stimulated, ABX CD4+ stimulated, ABX CD8+ stimulated, WT CD8+ stimulated',
 'WT CD4+ stimulated, ABX CD4+ stimulated, WT CD8+ stimulated, ABX CD8+ stimulated',
 'WT CD4+ stimulated, ABX CD8+ stimulated, ABX CD4+ stimulated, WT CD8+ stimulated',
 'WT CD4+ stimulated, ABX CD8+ stimulated, WT CD8+ stimulated, ABX CD4+ stimulated',
 'WT CD4+ stimulated, WT CD8+ stimulated, ABX CD4+ stimulated, ABX CD8+ stimulated',
 'WT CD4+ stimulated, WT CD8+ stimulated, ABX CD8+ stimulated, AB

In [ ]:
##row 5 - permutation try
        dbc.Row([dbc.Col(html.Div([
            html.H4("Select Permutation to Plot", style=centerStyle),
            dcc.Dropdown(
                id="dropdown_permutation_labels",
                options=permutations,
                value= default_permutation,
                clearable=False)]), 
                width={"size": 6, "offset": 3})]),

In [25]:
#this column creates a new column which combines all of the descriptions
df_['Treatment Conditions'] = df_[description_list].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1)

@app.callback(
    dash.dependencies.Output('ordered_opt', 'children'),
    [dash.dependencies.Input('permutations_list', 'value')])
def permutationToPlot (selected_permutation):
    x = selected_permutation.split(", ")
    df_['Treatment Conditions'] = pd.Categorical(df_['Treatment Conditions'], categories=x, ordered=True)
    df_.sort_values('Treatment Conditions')
    return df_

,Unnamed: 0,Donor,Cell Subset,Stimulation,BCA-1,CCL-11,FAS,GM-CSF,Granzyme B,IFN-g,...,IL-7,IP-10,KC,MCP-1,MIP-1a,RANTES,sCD137,TGF-b,TNF-a,Treatment Conditions
0,0,ABX,CD4+,stimulated,0.0,0.0,0.0,0.0,2.96,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,ABX CD4+ stimulated
327,327,ABX,CD4+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,ABX CD4+ stimulated
326,326,ABX,CD4+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,ABX CD4+ stimulated
325,325,ABX,CD4+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,ABX CD4+ stimulated
324,324,ABX,CD4+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,2.62,0.0,0.0,0.0,0.0,ABX CD4+ stimulated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2050,2050,ABX,CD8+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,ABX CD8+ stimulated
2051,2051,ABX,CD8+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,ABX CD8+ stimulated
2052,2052,ABX,CD8+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,ABX CD8+ stimulated
2046,2046,ABX,CD8+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,ABX CD8+ stimulated


In [9]:
import itertools
df_labels_un = list(df_['Treatment Conditions'].unique())
#this creates permutations of the list that the user selects to order the graph
permutations = list(itertools.permutations(df_labels_un))
permutations_list = []
for i in permutations:
    permutations_list.append(", ".join(i))
permutations_list

selected_permutation = permutations_list[8]
x = selected_permutation.split(", ")

print(selected_permutation)
print(x)

WT CD4+ stimulated, ABX CD8+ stimulated, ABX CD4+ stimulated, WT CD8+ stimulated
['WT CD4+ stimulated', 'ABX CD8+ stimulated', 'ABX CD4+ stimulated', 'WT CD8+ stimulated']


In [10]:
def permutationToPlot (selected_permutation):
    x = selected_permutation.split(", ")
    #this function reorders the dataframe so user can select which order to plot
    count = 0;
    for i in x:
        if count == 0:
            new_data = df_[df_["Treatment Conditions"] == i]
            count = count + 1
        else:
            df_to_append = df_[df_["Treatment Conditions"] == i]
            new_data = pd.concat([new_data, df_to_append])
            count = count + 1





,Unnamed: 0,Donor,Cell Subset,Stimulation,BCA-1,CCL-11,FAS,GM-CSF,Granzyme B,IFN-g,...,IL-7,IP-10,KC,MCP-1,MIP-1a,RANTES,sCD137,TGF-b,TNF-a,Treatment Conditions
480,480,WT,CD4+,stimulated,0.0,0.0,0.0,0.0,3.35,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WT CD4+ stimulated
481,481,WT,CD4+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WT CD4+ stimulated
482,482,WT,CD4+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WT CD4+ stimulated
483,483,WT,CD4+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WT CD4+ stimulated
484,484,WT,CD4+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WT CD4+ stimulated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3359,3359,WT,CD8+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WT CD8+ stimulated
3360,3360,WT,CD8+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WT CD8+ stimulated
3361,3361,WT,CD8+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WT CD8+ stimulated
3362,3362,WT,CD8+,stimulated,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WT CD8+ stimulated


In [11]:
secretome_selection = ['Mouse Adaptive Immune', 'Human Adaptive Immune', 'NHP Adaptive Immune', 
'Human Inflammation', 'Human Innate Immune']

#Select Secretome to use
mouse_adaptive_immune = ['BCA-1', 'CCL-11', 'FAS', 'GM-CSF', 'Granzyme B', 
    'IFN-g', 'IL-10', 'IL-12p70', 'IL-13', 'IL-15', 'IL-17A', 
    'IL-18', 'IL-1b', 'IL-2', 'IL-21', 'IL-27', 'IL-4', 
    'IL-5', 'IL-6', 'IL-7', 'IP-10', 'KC', 'MCP-1', 'MIP-1a', 
    'RANTES', 'sCD137', 'TGF-b', 'TNF-a']

human_adaptive_immune = ['Granzyme B', 'IFN-g', 'MIP-1a', 'Perforin', 'TNF-a',
    'TNF-b', 'GM-CSF', 'IL-2', 'IL-5', 'IL-7', 'IL-8', 'IL-9', 'IL-12', 'IL-15', 
    'IL-21', 'CCL-11', 'IP-10', 'MIP-1b', 'RANTES', 'IL-4', 'IL-10', 'IL-13',
    'IL-22', 'TGF-b1', 'sCD137', 'sCD40L', 'IL-1b', 'IL-6', 'IL-17a', 'IL-17f',
    'MCP-1', 'MCP-4']

nhp_adaptive_immune = ['TNF-a', 'MCP-1', 'IL-2', 'IL-4', 'MIP-1b', 'IL-6', 'IL-8',
    'IL-1b', 'RANTES', 'IFN-g', 'IP-10', 'MIP-1a', 'MIF', 'GM-CSF']

human_innate_immune = ['IFN-g', 'MIP-1a', 'TNF-a', 'TNF-b', 'Stimulatory', 'GM-CSF', 
    'IL-8', 'IL-9', 'IL-15', 'IL-18', 'TGF-a', 'IL-5', 'CCL11', 'IP-10', 'MIP-1b', 
    'RANTES', 'BCA-1', 'IL-10', 'IL-13', 'IL-22', 'sCD40L', 'IL-1b', 'IL-6', 
    'IL-12-p40', 'IL-12', 'IL-17a', 'IL-17f', 'MCP-1', 'MCP-4', 'MIF', 'EGF',
    'PDGF-BB', 'VEGF']

human_inflammation = ["GM-CSF", 'IFN-g', 'IL-2', 'IL-12', 'TNF-a', 'TNF-b', 'IL-4',
'IL-5', 'IL-7', 'IL-9', 'IL-13', 'CCL11', 'IL-8', 'IP-10', 'MCP-1', 'MCP-4', 'MIP-1a',
'MIP-1b', 'RANTES', 'IL-10', 'IL-15', 'IL-22', 'TGF-b1', 'IL-1b', 'IL-6', 'IL-17a', 
'IL-17f', 'IL-21', 'Granzyme B', 'Perforin', 'sCD40L', 'sCD137']

dcc.RadioItems(
        id="secretome_selection",
        options=secretome_selection,
        value= "Mouse Adaptive Immune",
        inline=True, inputStyle={"margin-right": "5px", "margin-left": "5px"}, 
        style=centerStyle), 

In [1]:
def cyto_secretion_list(val):
    if val == "Mouse Adaptive Immune":
        cyto_list = ['BCA-1', 'CCL-11', 'FAS', 'GM-CSF', 'Granzyme B', 
        'IFN-g', 'IL-10', 'IL-12p70', 'IL-13', 'IL-15', 'IL-17A', 
        'IL-18', 'IL-1b', 'IL-2', 'IL-21', 'IL-27', 'IL-4', 
        'IL-5', 'IL-6', 'IL-7', 'IP-10', 'KC', 'MCP-1', 'MIP-1a', 
        'RANTES', 'sCD137', 'TGF-b', 'TNF-a']
    elif val  == 'Human Adaptive Immune':
        cyto_list = ['Granzyme B', 'IFN-g', 'MIP-1a', 'Perforin', 'TNF-a',
        'TNF-b', 'GM-CSF', 'IL-2', 'IL-5', 'IL-7', 'IL-8', 'IL-9', 'IL-12', 'IL-15', 
        'IL-21', 'CCL-11', 'IP-10', 'MIP-1b', 'RANTES', 'IL-4', 'IL-10', 'IL-13',
        'IL-22', 'TGF-b1', 'sCD137', 'sCD40L', 'IL-1b', 'IL-6', 'IL-17a', 'IL-17f',
        'MCP-1', 'MCP-4']
    elif val == 'NHP Adaptive Immune':
        cyto_list = ['TNF-a', 'MCP-1', 'IL-2', 'IL-4', 'MIP-1b', 'IL-6', 'IL-8',
        'IL-1b', 'RANTES', 'IFN-g', 'IP-10', 'MIP-1a', 'MIF', 'GM-CSF']
    elif val == 'Human Inflammation':
        cyto_list = ["GM-CSF", 'IFN-g', 'IL-2', 'IL-12', 'TNF-a', 'TNF-b', 'IL-4',
        'IL-5', 'IL-7', 'IL-9', 'IL-13', 'CCL11', 'IL-8', 'IP-10', 'MCP-1', 'MCP-4', 'MIP-1a',
        'MIP-1b', 'RANTES', 'IL-10', 'IL-15', 'IL-22', 'TGF-b1', 'IL-1b', 'IL-6', 'IL-17a', 
        'IL-17f', 'IL-21', 'Granzyme B', 'Perforin', 'sCD40L', 'sCD137']
    else:
        cyto_list = ['IFN-g', 'MIP-1a', 'TNF-a', 'TNF-b', 'Stimulatory', 'GM-CSF', 
        'IL-8', 'IL-9', 'IL-15', 'IL-18', 'TGF-a', 'IL-5', 'CCL11', 'IP-10', 'MIP-1b', 
        'RANTES', 'BCA-1', 'IL-10', 'IL-13', 'IL-22', 'sCD40L', 'IL-1b', 'IL-6', 
        'IL-12-p40', 'IL-12', 'IL-17a', 'IL-17f', 'MCP-1', 'MCP-4', 'MIF', 'EGF',
        'PDGF-BB', 'VEGF']
    return cyto_list
    

In [7]:
def col_row_check (n, cyto_list, df):
    if n is None: 
        raise PreventUpdate
    else:
        df = pd.DataFrame(df)
        df_sub = df[cyto_list].T
        row, col = df_sub.shape           
        numCytokines = 'Number of Cytokines: ' + str(row)
        numCells = 'Number of Cells: ' + str(col)
        iso_stats_header = 'Isoplexis Data Analysis'
        return(numCytokines, numCells, iso_stats_header)

col_row_check(1, cyto_list, df_)

('Number of Cytokines: 28', 'Number of Cells: 3364', 'Isoplexis Data Analysis')

In [65]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def pca_func(n, method, plot_type, cytokines, df):#, color_discrete_map):
    if n is None: 
        #return no_update
        return ("ok")
    else:
        df = pd.DataFrame(df)
        Y = df.loc[:,['Treatment Conditions']].values
        x = df.loc[:, cytokines].values
        # # Separating out the target
        if method == "Standard Scalar Normalized":
             x = StandardScaler().fit_transform(x)
        pca = PCA(n_components=3)
        principalComponents = pca.fit_transform(x)
        df = pd.DataFrame(principalComponents)
        df2 = df.rename({0: "PCA 1", 1: "PCA 2", 2: "PCA 3"}, axis=1) 
        df2['Treatment Conditions'] = Y
        if plot_type == "2D":
            fig = px.scatter(df2, x="PCA 1", y="PCA 2", color="Treatment Conditions") #, color_discrete_map = color_discrete_map)
        else:
            fig = px.scatter_3d(df2, x="PCA 1", y="PCA 2", z="PCA 3", color="Treatment Conditions") #color_discrete_map = color_discrete_map)
        fig.update_layout(title_text = method+" PCA", title_x=0.5, )
        fig.update_layout(plot_bgcolor='rgb(255,255,255)')
        return(fig, df2)

fig, df2 = pca_func(1, "Standard Scalar Normalized", "3D", cyto_list, df_)
df2



,PCA 1,PCA 2,PCA 3,Treatment Conditions
0,0.071136,-0.051674,0.026207,ABX CD4+ stimulated
1,-0.091969,-0.017752,-0.234001,ABX CD4+ stimulated
2,-0.091969,-0.017752,-0.234001,ABX CD4+ stimulated
3,-0.091969,-0.017752,-0.234001,ABX CD4+ stimulated
4,-0.091969,-0.017752,-0.234001,ABX CD4+ stimulated
...,...,...,...,...
3359,-0.091969,-0.017752,-0.234001,WT CD8+ stimulated
3360,-0.091969,-0.017752,-0.234001,WT CD8+ stimulated
3361,-0.091969,-0.017752,-0.234001,WT CD8+ stimulated
3362,-0.091969,-0.017752,-0.234001,WT CD8+ stimulated


In [60]:
import plotly.express as px

print(px.colors.sequential.thermal)

['rgb(3, 35, 51)', 'rgb(13, 48, 100)', 'rgb(53, 50, 155)', 'rgb(93, 62, 153)', 'rgb(126, 77, 143)', 'rgb(158, 89, 135)', 'rgb(193, 100, 121)', 'rgb(225, 113, 97)', 'rgb(246, 139, 69)', 'rgb(251, 173, 60)', 'rgb(246, 211, 70)', 'rgb(231, 250, 90)']


In [74]:
def tsne_func(n, method, plot_type, perplexity, iterations, cytokines, df , color_discrete_map):
    if n is None: 
        #return no_update
        return("ok")
    else:
        df = pd.DataFrame(df)
        Y = df.loc[:,['Treatment Conditions']].values
        x = df.loc[:, cytokines].values
        # # Separating out the target
        if method == "Standard Scalar Normalized":
             x = StandardScaler().fit_transform(x)
        pca = PCA()
        principalComponents = pca.fit_transform(x)
        tsne = TSNE(n_components=3, perplexity=perplexity, n_iter=iterations).fit_transform(principalComponents)
        df = pd.DataFrame(tsne)
        df2 = df.rename({0: "TSNE 1", 1: "TSNE 2", 2: "TSNE 3"}, axis=1) 
        df2['Treatment Conditions'] = Y
        if plot_type == "2D":
            fig = px.scatter(df2, x="TSNE 1", y= "TSNE 2", color="Treatment Conditions", color_discrete_map = color_discrete_map)
        else:
            fig = px.scatter_3d(df2, x="TSNE 1", y="TSNE 2", z="TSNE 3", color="Treatment Conditions", color_discrete_map = color_discrete_map)      
        fig.update_layout(title_text = method + " TSNE", title_x=0.5, )
        fig.update_layout(plot_bgcolor='rgb(255,255,255)')
        return(fig)

tsne_func(1, "Standard Scalar Normalized", "3D", 30, 500, cyto_list, df_)

/home2/s180020/.conda/envs/isoplexis/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home2/s180020/.conda/envs/isoplexis/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

